# 强化学习2: 动态规划与迭代法
* 0.强化学习中如何用到动态规划与迭代法？
* 1.动态规划
* 2.迭代法
* 3.策略迭代与价值迭代

<img src='RL_1.png',width=600>



* Flappy bird的简单解决方法
    * 如何衡量总价值
    * 如何选择动作，选择总策略
* **总价值不易计算时，但环境状态有显式的分布时**
    * 如何使用迭代法计算总价值
    * 如何使用迭代法反复改进总策略
    * 策略迭代法的收敛
* 总价值不易计算时，环境状态没有显式的分布时，从连续的样本和经验中学习
    * 蒙特卡洛方法
    * 计算总价值
    * 更新总策略
* 总价值不易计算时，环境状态没有显式的分布时，从每一次与环境状态的交互中学习
    * Temporal Differences
    * Temporal Differences与蒙特卡罗方法的对比
    * SARSA
    * Q-learning
* 当环境状态过多，如何将有限样本中的策略推广到更大的状态空间，作为更大状态空间的近似解？
    * 结合监督学习
    * 线性方法等
* Q-learing+Deep-Learning
    * DQN
    * DQN的优势与特点


[参考视频](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html)

[参考书籍](https://www.amazon.com/Reinforcement-Learning-Introduction-Adaptive-Computation/dp/0262193981)

[参考中文知乎](https://www.zhihu.com/people/flood-sung/activities)

# 0.动态规划与迭代法用在哪里？

### 0.1 在计算策略总价值时
* 当完全知道环境的动态变化，但计算不方便时
<img src='v_pi.png',width=500>

* 采用近似算法计算在策略$\pi$下的总价值
<img src='v_k.png',width=600>
* 最终计算而得的不动点，就应该是$v_\pi(s)$
* 这个步骤，叫做 **iterative policy evaluation 迭代法求策略总价值**

### 0.2 在更新策略，求最优策略时
* 迭代法求最优策略
* 求对应策略总价值 -> 更新策略 -> 求对应策略总价值 -> 更新策略
* GPI

# 复习动态规划

1. 从硬币凑数开始
    * 动态规划的特点--分而治之
     
2. 斐波拉契数列
    * 递归解法
    * BottomUp
    * TopDown+memory
    * 比递归好在哪里？
        * 拿空间换时间
        
3. 最长递增数列
4. 最短路径
5. Value/Policy Iteration

## 1. 凑硬币

现有硬币若干，面值为1分硬币，2分硬币，5分硬币，如果用最少的硬币个数凑出要求的金额？

分析题目：
1. 解决的问题是什么？总额=[1分，2分，3分，4分。。。。。1元，1元1分，1元2分，1元3分，1元4分，1元5分。。。。]
2. 对某个总额的最优解要求是什么？
3. 如何达到这个最优解？是否可借助其他总额的最优解？
    

正向的求解过程
<img src='DP_coins_3.png',width=700>

逆向的求解过程
<img src='DP_coins_2.png',width=700>

## DP的重点：分解成关联的子问题

** 分而治之  ** + **储存结果**

## 2. 斐波拉契数列

* 递归解法
* BottomUp
* TopDown+memory
* 动态规划解法比递归好在哪里？
    * **空间换时间**

In [ ]:
# 递归的斐波拉契
def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1

    return fib(n - 1) + fib(n - 2)

In [2]:
# Bottom Up 
cache = {}

def fib(n):
    cache[0] = 0
    cache[1] = 1

    for i in range(2, n + 1):
        cache[i] = cache[i - 1] +  cache[i - 2]

    return cache[n]

In [1]:
# TopDown+memory
cache = {} # dictionary

def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    if n in cache:
        return cache[n]

    cache[n] = fib(n - 1) + fib(n - 2)

    return cache[n]

### 动态规划解法比递归好在哪里？
fib(n) = fib(n-1)+fib(n-2)

fib(n-1) = fib(n-2)+fib(n-3)

fib(n-2) = fib(n-3)+fib(n-4)

......

<img src='fib.png',width=700>

## 3. 最长递增序列


<img src='DP_0nn1.png',width=800>

* 问题描述： 找出数列A中的最长递增序列的长度
* 几个变量：
    * i
    * A[i]
    * 长度S(i)
    * 最长序列倒数第二的元素位置

<img src='DP_0nn2.png',width=800>

<img src='DP_0nn3.png',width=800>

In [4]:
def DP_nn_S(A):
    """这个解法比较直观，简单，复杂度O(n^2)."""

    S = []
    D2 = [] # 重建最长序列时用
    Longest_end = 0 #最佳的序号，做记录用

    for current in range(len(A)):
        # 第一个元素组成的序列的最长递增长度为1            
        S.append(1)

        # 最后开始这个元素没有 倒数第二元素             
        D2.append(None)

        for prev_ele in range(current):
            # 如果之前的元素比当前的小，并且以之前元素为结尾的最长递增序列长度+1 比当前的长             
            if ((A[prev_ele] < A[current]) and (S[prev_ele] + 1) > S[current]):
                # 把当前元素接在之前元素的最长递增序列末尾，像链表一样，改变当前序列的长度    
                S[current] = S[prev_ele] + 1
                D2[current] = prev_ele

        if (S[current] > S[Longest_end]):
            Longest_end = current

    return S[Longest_end]

* 另一种DP解法从不同的角度出发
[wiki](https://en.wikipedia.org/wiki/Longest_increasing_subsequence)

<img src='DP_nlogn.png',width=800>

<img src='LISDemo.gif',width=400>

In [ ]:
import numpy as np
def DP_nlogn(X):
    # 复杂度 nlog(n)
    N = len(X)
    # P[i]是用来记录X[i]在已X[i]为末尾的递增序列中的倒数第二个元素的序号
    # 可以通过P[i], P[P[i]],P[P[P[i]]]，...，把递增序列找出来
    P = np.zeros(N)
    M = np.zeros(N+1)

    L = 0
    for i in range(N):
        # 用二分法在生成的递增序列中找到比X[i]小的最大数的序号+1
        lo = 1
        hi = L
        while lo < hi:
            mid = ceil((lo+hi)/2)
            if X[M[mid]] < X[i]:
                lo = mid+1
            else:
                hi = mid-1
        # 将这个值赋值给newL
        newL = lo

        # The predecessor of X[i] is the last index of 
        # the subsequence of length newL-1
        # 找到的子递增序列中，X[i]的前面一个元素的序号是M[new-1]，也就是P[i]。
        # P[i]是用来记录X[i]在已X[i]为末尾的递增序列中的倒数第二个元素的序号
        
        P[i] = M[newL-1]
        M[newL] = i

        if newL > L:
        # 如果新的序列比以前的长度长，说明X[i]比以前的递增序列最末元素的数值大
        # 更新序列长度
            L = newL

    # 重建序列
    S = np.zeros(N)
    k = M[L-1]
    for i in range(L-1,-1,-1):
        S[i] = X[k]
        k = P[k]

    return S

# 迭代法
* 1.数值计算的基本方法：迭代法解线性方程组
    * Gauss消去法的复杂度是n3，迭代法复杂度是n2
* 2.Jacob方法
* 3.Gauss-Seidel方法
* 4.收敛性

### 2. Jacob方法

\begin{equation}
9x_1 + x_2 + x_3 = b_1\\
2x_1 + 10x_2 + 3x_3 = b_2\\
3x_1+4x_2+11x_3 = b_3\\
\end{equation}

* 变形
\begin{equation}
x_1 = 1/9 [b_1 - x_2-x_3]\\
x_2 = 1/10 [b_2 -2x_1-3x_3]\\
x_3 = 1/11[b_3-3x_1-4x_2]\\
\end{equation}

* 初始化 $ x^(0) = [x_1^{(0)},x_2^{(0)},x_3^{(0)}]^T$

* 反复迭代
* 变形
\begin{equation}
x_1^{(k+1)} = 1/9 [b_1 - x_2^{(k)}-x_3^{(k)}]\\
x_2^{(k+1)} = 1/10 [b_2 -2x_1^{(k)}-3x_3^{(k)}]\\
x_3^{(k+1)} = 1/11[b_3-3x_1^{(k)}-4x_2^{(k)}]\\
\end{equation}

* 例如，当b=$[10,19,0]^T$时，精确解应为： $x=[1,2,-1]^T$
* 利用Jacob方法

In [23]:
def jacob_iter(x,b):
    x_new = [0,0,0]
    x_new[0]=1/9*(b[0]-x[1]-x[2])
    x_new[1]=1/10*(b[1]-2*x[0]-3*x[2])
    x_new[2]=1/11*(b[2]-3*x[0]-4*x[1])
    return x_new

In [24]:
x=[0,0,0]
b=[10,19,0]
for i in range(20):
    x=jacob_iter(x,b)
    print("%f,%f,%f"%(x[0],x[1],x[2]))

1.111111,1.900000,0.000000
0.900000,1.677778,-0.993939
1.035129,2.018182,-0.855556
0.981930,1.949641,-1.016192
1.007395,2.008472,-0.976760
0.996476,1.991549,-1.005097
1.001505,2.002234,-0.995966
0.999304,1.998489,-1.001223
1.000304,2.000506,-0.999260
0.999862,1.999717,-1.000267
1.000061,2.000108,-0.999859
0.999972,1.999946,-1.000056
1.000012,2.000022,-0.999973
0.999994,1.999989,-1.000011
1.000002,2.000005,-0.999995
0.999999,1.999998,-1.000002
1.000000,2.000001,-0.999999
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000


### 3. Gauss-Seidel迭代法

\begin{equation}
9x_1 + x_2 + x_3 = b_1\\
2x_1 + 10x_2 + 3x_3 = b_2\\
3x_1+4x_2+11x_3 = b_3\\
\end{equation}

* 变形
\begin{equation}
x_1 = 1/9 [b_1 - x_2-x_3]\\
x_2 = 1/10 [b_2 -2x_1-3x_3]\\
x_3 = 1/11[b_3-3x_1-4x_2]\\
\end{equation}

* 反复迭代
* 变形
\begin{equation}
x_1^{(k+1)} = 1/9 [b_1 - x_2^{(k)}-x_3^{(k)}]\\
x_2^{(k+1)} = 1/10 [b_2 -2x_1^{(k+1)}-3x_3^{(k)}]\\
x_3^{(k+1)} = 1/11[b_3-3x_1^{(k+1)}-4x_2^{(k+1)}]\\
\end{equation}

In [20]:
def gs_iter(x,b):
    x_new = [0,0,0]
    x_new[0]=1/9*(b[0]-x[1]-x[2])
    x_new[1]=1/10*(b[1]-2*x_new[0]-3*x[2])
    x_new[2]=1/11*(b[2]-3*x_new[0]-4*x_new[1])
    return x_new

In [22]:
x=[0,0,0]
b=[10,19,0]
for i in range(20):
    x=gs_iter(x,b)
    print("%f,%f,%f"%(x[0],x[1],x[2]))

1.111111,1.677778,-0.913131
1.026150,1.968709,-0.995753
1.003005,1.998125,-1.000138
1.000224,1.999997,-1.000060
1.000007,2.000017,-1.000008
0.999999,2.000003,-1.000001
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000
1.000000,2.000000,-1.000000


### 4. 收敛性
* 对方程组Ax=b
* $Nx^{(k+1)}=b+Px^{k}$
* Jacob方法：
    * N为对角线矩阵，P为N-A
* G-S方法
    * N为下三角矩阵，P为N-A
* $N(x-x^{(k+1)}=P(x-x^{(k)})$
* $e^{(k+1)}=N^{-1}Pe^{(k)}$

# 迭代法求策略估值与迭代法更新最佳策略

### 1.迭代法求策略估值

<img src='v_pi.png',width=500>

<img src='v_k.png',width=500>

<img src='policy_ev_alg.png',width=700>

* 例子
<img src='policy_ev.png',width=600>

<img src='policy_ev_table.png',width=300>

### 2.迭代法更新最佳策略

<img src='policy_iter.png',width=600>

* 例子1：
<img src='policy_improve_1.png',width=400>

* 例子2:
* 老贾掌管两个租车行A,B，有车的时候只要有单，就可赚10元。老贾每晚可以在两个租车行间调配第二天的备用车辆数量。每移动一辆，花费2元。已知每天租车和还车的客人成泊松分布$\frac{e^{-\lambda}\lambda^{n}}{n!}$。对每个车行，每天来租车和还车的泊松分布参数分别为租车A,B = 3,4, 还车A,B = 3,2. 每个车行最多有20辆后备车。$\gamma$=0.9，求每晚老贾在A,B间调配车辆的策略。

<img src='policy_improve_2.png',width=700>

* 为什么持续迭代更新旧的策略可以获得更好的策略？
* 命题：如果$q_{\pi}(s,\pi'(s))>=v_{\pi}(s)$，那么$v_{\pi'}(s)>=v_{\pi}(s)$

<img src='policy_improve_3.png',width=500>